In [1]:
import numpy as np;
from math import sin, cos, sqrt, atan2, radians,inf;
import random; 
from apscheduler.schedulers.background import BackgroundScheduler as scheduler
import apscheduler

class Autocar:   
    def __init__(self,AC_location = 0,state_of_charge = -1,min_lat = 52,
                 min_long = 12.9,capacity = 75, powerpdist = 14):
        #map constraint
        self.AC_location = np.random.random(2) + (min_lat,min_long)
        self.min_lat = min_lat;
        self.min_long = min_long;
        # car representet as random distribution funktion --> random langitude and latidude
        #AC_long = np.random.random(1);
        #AC_lat = np.random.random(1);
        # alternative: random function generates random vector that represent a car with only 20% of charge
        # random number amount between 0 and 1
        # car needs parameters
        self.capacity  = capacity; # example for Tessla S modal, BMWi3 ; smart60kW fortwo 17,6kWh
        #possible_distance = 400 # Tessla s Modal; 260km BMWi3 (daily distance)
        #Stromverbrauch in kWh/100 km: 14,6 - 14,0
        self.powerpdist = powerpdist;#power per distance --> kWh per 100km
        #powtodist = powerpdist/100; #power [kWh] need for 1km; theoretically something that might be measured in the car andoptimized by the car itself
        self.powerkm = powerpdist/100
        if state_of_charge < 0:    
            self.state_of_charge = round(random.uniform(0.2,1), 3); # state of battery between 0 and 1 --> if it is smaller 0.2 it will check where it can charge its battery
        else:
            self.state_of_charge = state_of_charge;
            
    def show_attributes(self):
        print('location:',self.AC_location,' (min_lat:', self.min_lat,'min_long:', self.min_long,')\ncapacity [kWh]:',
              self.capacity,'\npower consumption per km:', self.powerkm,'\nstate of charge [% of capacity]:',self.state_of_charge*100 )

    def find_distance_km(self, lat1,lng1,lat2,lng2):
        EARTH_RADIUS = 6373.0
        lat1 = radians(lat1)
        lng1 = radians(lng1)
        lat2 = radians(lat2)
        lng2 = radians(lng2)
        
        dlon = lng2 - lng1
        dlat = lat2 - lat1
        
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        
        return EARTH_RADIUS * c 
    
class Grid:   
    def __init__(self, grid_location = 0, total_charge_needed_at_grid=0, dist=0, price=0, alpha = 0, name = "default"):
        #map constraint
        self.grid_location = np.random.random(2) + (min_lat,min_long)
        self.total_charge_needed_at_grid = total_charge_needed_at_grid
        self.dist = dist
        self.price = price
        self.name = name
        self.alpha = random.uniform(0,1)


In [2]:
#############################
### linear Programming#######
#############################

def optimize(grids, mode):

    prob = LpProblem("AC", LpMinimize)
    variables = []
    
    ### every x represents a boolean variable wether to pick a grid
    for x in range(len(grids[0:3])):
            variables.append(LpVariable(str(x), 0,1, LpInteger))
            
    ### constraints
    if mode=="eco_mode":
        prob += grids[0].dist*variables[0]*grids[0].alpha+grids[1].dist*variables[1]*grids[1].alpha+grids[2].dist*grids[2].alpha*variables[2]
    if mode=="costSaving_mode":
            prob += grids[0].price*grids[0].total_charge_needed_at_grid*variables[0]+grids[1].price*grids[1].total_charge_needed_at_grid*variables[1]+grids[2].price*grids[2].total_charge_needed_at_grid*variables[2]

    prob += variables[0]+variables[1]+variables[2] == 1

### CONSTRAINTS
# all binary decision variables should sum up to 1
    #prob += grid_1+grid_2+grid_3 == 1

    prob.writeLP("AC.lp")
    prob.solve()

    for x in range(len(variables)):
        if(value(variables[x])==1):
        #print("all grid locations:", grids_loc_final[0:3])
            #print("all grid distances:", grids[0:3])
            #print("Lowest distance to..", grids[x].name)
            #print("Lowest distance/price: ", value(prob.objective))
            print("Driving to grid at... ", grids[x].grid_location)
            print("Car fully charged! 100% battery!")
            return grids[x].grid_location
    


In [3]:
import random
import pandas as pd
from pulp import * 

min_lat = 52
min_long = 12.9


def create_vehicles(num):
    acs = []
    for x in range(num):
        acs.append(Autocar(capacity = random.uniform(50,120), powerpdist = random.uniform(5, 20)))
    return acs

def create_grids(num):
    grids_loc = []
    for x in range(num):
        grids_loc.append(Grid(name="grid "))
    return grids_loc

def reachable_grids(ac, grids):
    final_grids = []
    for x in range(len(grids)):
        ### calculate total energy to grid considering distance
        grids[x].dist = ac.find_distance_km(grids[x].grid_location[0], grids[x].grid_location[1], ac.AC_location[0], ac.AC_location[1])
        c_to_grid = ac.powerkm*grids[x].dist
        ### consider only reachable grids
        if (c_to_grid <= (ac.state_of_charge*ac.capacity)):
            ### add the energy needed to reach grid to the total deficit
            grids[x].total_charge_needed_at_grid = c_to_grid+(ac.capacity-(ac.state_of_charge*ac.capacity))
            #energy_deficit.append(total_charge[0])
            grids[x].price = random.uniform(0.4,0.8)
            ### location, total energy needed at grid location, distance, price
            final_grids.append(grids[x])
    return final_grids

def drive_vehicle(ac, grids, modus):
        # assuming v = 50km/h
        soc = random.uniform(0.4,1)
        code = 0
        grid_loc = [0,0]
        ### available energy
        powerstate = ac.state_of_charge*ac.capacity
        ### needed energy
        consumption = 0.9*ac.powerpdist
        if (powerstate-consumption > 0):
            ### available energy - needed energy
            powerupdate = powerstate-consumption
            ### percentage battery left (new state of charge)
            soc_update = round((powerupdate/ac.capacity), 2)
            if (soc_update <= 0):
                print(soc_update)
                print("Battery below 1%.. Randomly resetting SOC to ", soc)
            if (soc_update <= 0.2):
                print("Searching for a charging station..")
                #eco_mode(grids)
                ########################################
                ##### Change Optimization Goal Here ####
                ########################################
                grid_loc = optimize(grids,  modus)
                soc = 1.0
                ### code for update within reach
                code = 1
            else:
                soc = soc_update
        else:
            print("Car out of power.. Randomly resetting SOC to ", soc)
        ac.state_of_charge = soc
        # return the code for weather to update the location of the car, the current soc and the chosen grid_loc
        return [code, soc, grid_loc]


In [4]:
def main(numAC, numGrids, modus):
    print("Enter 'b' for stopping and 's' for starting the vehicle")
    ### Create vehicles
    vehicles = create_vehicles(numAC)
    ### Create grids
    grids = create_grids(numGrids)
    ### choose one car to work with 
    ac = vehicles[0]
    ### Find grids which can be reached with current state of charge
    grids_within_reach = reachable_grids(ac, grids)
    
    ### scedule Loop
    sched = scheduler()
    sched.start()

    # scheuler listener, listenes to scheduler events
    def listener(event):
        # !!!! successfull iteration of the job !!!!
        if(event.code == 4096):
        ###########################################
        # store and print updated State of Charge ##
        ###########################################
            current_SOC = event.retval[1]
            print("CURRENT SOC: ", current_SOC)
        
            if (event.retval[2][0] != 0):
                    ########################################################
                    ##### location of chosen grid during optimization ######
                    ########################################################
                    chosen_grid_location = event.retval[2]
                    #print("WENT TO GRID AT... ", event.retval[2])
                
                # update location if soc was charged to 100%
            if event.retval[0]==1:
                # randomly update location after charging process
                ac.AC_location = np.random.random(2) + (min_lat,min_long)
                        
                # update grids within reach according to new ac location
                grids_within_reach = reachable_grids(ac, grids)
                
    sched.add_job(drive_vehicle, 'interval', args=[ac,grids_within_reach, modus], seconds=1)
    sched.add_listener(listener)
    
    while True:
        if input() == 'b':
            #sched.shutdown(wait=False)
            sched.pause()
            print("Stopping Operation. Shutting down vehicle...")
        if input() == 's':
            sched.resume()
            print("Starting Operation. Powering up vehicle...")






In [ ]:
#  enter number ACs and number Grids
main(3, 6, "costSaving_mode")

Enter 'b' for stopping and 's' for starting the vehicle
CURRENT SOC:  0.57
CURRENT SOC:  0.25
Car out of power.. Randomly resetting SOC to  0.8320784019209959
CURRENT SOC:  0.8320784019209959
CURRENT SOC:  0.52
Searching for a charging station..
Driving to grid at...  [52.30739207 13.01477413]
Car fully charged! 100% battery!
CURRENT SOC:  1.0
CURRENT SOC:  0.68
CURRENT SOC:  0.36
Searching for a charging station..
Driving to grid at...  [52.27124058 13.57313012]
Car fully charged! 100% battery!
CURRENT SOC:  1.0
CURRENT SOC:  0.68
CURRENT SOC:  0.36
Searching for a charging station..
Driving to grid at...  [52.30739207 13.01477413]
Car fully charged! 100% battery!
CURRENT SOC:  1.0
CURRENT SOC:  0.68
CURRENT SOC:  0.36
Searching for a charging station..
Driving to grid at...  [52.0679476  13.40452987]
Car fully charged! 100% battery!
CURRENT SOC:  1.0
CURRENT SOC:  0.68
CURRENT SOC:  0.36
Searching for a charging station..
Driving to grid at...  [52.27124058 13.57313012]
Car fully cha